In [1]:
import glob
import time
import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import bezpy

In [2]:
from astropy.time import Time
import datetime
isotfmt = '%Y-%m-%dT%H:%M:%S.%f'
def MJD2UT(mjd):
    """ If given single value, will return single datetime.datetime
    If given list, will return list of datetime.datetimes
    """
    UT = Time(mjd, format='mjd').isot
    if type(UT) == str:
        return datetime.datetime.strptime(UT,isotfmt)
    else:
        return [datetime.datetime.strptime(UT[n],isotfmt) for n in range(len(UT))]


def read_mage(fname):
    """Read the given MAGE h5 file"""
    # The file of magnetic field data
    # f = h5py.File("mageDB_SM.h5", "r")
    # f = h5py.File("booDB.deltab.h5", "r")
    f = h5py.File(fname, "r")
    
    ntimes = len([x for x in f.keys() if "Step" in x])
    # Create the names manually to avoid sorting on non-zero padded strings
    steps = [f"Step#{i}" for i in range(ntimes)]

    # Calculate datetimes from the attributes
    times = [MJD2UT(f[step].attrs["MJD"]) for step in steps]

    # Set up the arrays
    lons = np.rad2deg(f["Phicc"][0, ...]).ravel()
    lons[lons > 180] -= 360
    lats = 90 - np.rad2deg(f["Thetacc"][0, ...]).ravel()
    nsites = len(lons)
    
    dB = np.empty((ntimes, nsites, 2))
    for i, step in enumerate(steps):
        dB[i, :, 0] = -f[step]["dBt"][0, ...].ravel()  # Theta [colatitude] (X)
        dB[i, :, 1] = f[step]["dBp"][0, ...].ravel()  # Phi (Y)

    ds = xr.Dataset(coords={"longitude": ("site", lons),
                          "latitude": ("site", lats),
                          "component": ["X", "Y"],
                          "time": times},
                  data_vars={"B": (("time", "site", "component"), dB)})
    return ds

In [3]:
def read_swmf(dirname):
    """Read the SWMF output contained in the given directory"""
    fnames = sorted(glob.glob(f"{dirname}/*.out"))
    ntimes = len(fnames)
    
    # Read in the first file to get the shapes
    test = pd.read_csv(fnames[0], delim_whitespace=True, header=3, usecols=[0, 1])
    
    npoints_swmf = len(test)
    lons = test["Lon"].to_numpy()
    lons[lons > 180] -= 360
    lats = test["Lat"].to_numpy()
    dB = np.empty((ntimes, npoints_swmf, 2))
    times = []

    # Now iterate over all the files
    for i, fname in enumerate(fnames):
        # Parse the filename for the date/time information
        time = datetime.datetime.strptime(fname[-19:-6], "%Y%m%d-%H%M")
        times.append(time)
        df = pd.read_csv(fname, delim_whitespace=True, header=3, usecols=[2, 3])
        dB[i, :, 0] = df["dBn"].to_numpy()
        dB[i, :, 1] = df["dBe"].to_numpy()
        
    ds = xr.Dataset(coords={"longitude": ("site", lons),
                          "latitude": ("site", lats),
                          "component": ["X", "Y"],
                          "time": times},
                  data_vars={"B": (("time", "site", "component"), dB)})
    return ds

# Reading in Impedance Data

In [4]:
def read_anna_file(fname):
    model_sites = {}

    with open(fname, 'r') as f:
        # 10 lines of header... Can this change?
        # YES! 0.25 degree files have 13
        for i in range(13):
            f.readline()

        for line in f:
            elements = line.split()
            period = float(elements[0])
            name = elements[1]
            lat = float(elements[2])
            lon = float(elements[3])
            component = elements[7]
            val = float(elements[8]) + float(elements[9])*1j

            if name not in model_sites:
                site = bezpy.mt.Site3d(name)
                model_sites[name] = site
                site.latitude = lat
                site.longitude = lon
                # 45 periods in the dataset
                site.periods = np.zeros(45)
                site.Z = np.zeros((4, 45), dtype=complex)
                old_period = 0.
                period_counter = -1

            if period != old_period:
                period_counter += 1
                old_period = period
                site.periods[period_counter] = period

            if component == 'ZXX':
                loc = 0
            elif component == 'ZXY':
                loc = 1
            elif component == 'ZYX':
                loc = 2
            elif component == 'ZYY':
                loc = 3

            site.Z[loc,period_counter] = val
            
    # Calculate resistivity before returning
    [site.calc_resisitivity() for site in model_sites.values()]
        
    return model_sites

In [5]:
def create_mapping(impedance_sites, ds):
    """Create a mapping from impedance sites to magnetic field location"""
    impedance_xys = np.array([(x.longitude, x.latitude) for x in impedance_sites])
    # Store a mapping from k -> (i, j)
    # impedance site location (k) -> magnetic field location (i, j)
    # NOTE: The values are shifted by 1/8 degree due to cell centered vs edges
    #       between the two datasets
    mapping_dict = {}
    for k in range(len(impedance_xys)):
        arr_i = np.argmin(abs(impedance_xys[k, 0] - ds["longitude"].to_numpy()) + abs(impedance_xys[k, 1] - ds["latitude"].to_numpy()))
        mapping_dict[k] = arr_i
    return mapping_dict

In [6]:
def calc_e(mapping_dict, b_ds):
    """Calculate the electric field at the impedance locations."""
    b_model = b_ds["B"].to_numpy()
    # Create an empty Electric field dataset that we can index into
    ntimes = b_model.shape[0]
    # We actually want B/E to be on the impedance site grid, not the original
    # model grid
    b = np.zeros((ntimes, nsites, 2))
    e = np.zeros_like(b)

    lons = []
    lats = []
    for site_num, site in enumerate(model_sites_sorted):
        lons.append(site.longitude)
        lats.append(site.latitude)
        # Pull out the magnetic field components for this site
        i = mapping_dict[site_num]
        b[:, site_num, 0], b[:, site_num, 1] = b_model[:, i, 0], b_model[:, i, 1]
        e[:, site_num, 0], e[:, site_num, 1] = site.convolve_fft(b[:, site_num, 0], b[:, site_num, 1], dt=60)
        
    # Create a new dataset
    ds = xr.Dataset(coords={"longitude": ("site", lons),
                          "latitude": ("site", lats),
                          "component": ["X", "Y"],
                          "time": b_ds["time"]},  # Same as model times
                  data_vars={"B": (("time", "site", "component"), b),
                             "E": (("time", "site", "component"), e)})
    return ds

In [7]:
fname = 'USA_impedance_gridded_45per_0.25x0.25-SP2.dat'
model_sites_all = read_anna_file(fname)

model_sites = {}
for name in model_sites_all:
    site = model_sites_all[name]
    model_sites[name] = site

del model_sites_all
model_sites_sorted = sorted(model_sites.values(), key=lambda x: (x.longitude, x.latitude))
nsites = len(model_sites_sorted)
print(f"There are {nsites} model points")
site_xys = np.array([(x.longitude, x.latitude) for x in model_sites_sorted])

There are 17765 model points


## Transmission lines

In [8]:
import geopandas as gpd
import pandas as pd


def get_transmission_lines():
# pickle delaunay calculations which is often a speed bottleneck
    try:
        return pd.read_pickle("transmission_line_objects.pkl")
    except FileNotFoundError:
        # Only recalculate if the weights haven't been filled before
        print("No transmission line pickle files found, loading the data"
              "and calculating values manually which can be slow")
        
    df = gpd.read_file("Electric_Power_Transmission_Lines.shp")
    # Change all MultiLineString into LineString objects by grabbing the first line
    # Will miss a few coordinates, but should be OK as an approximation
    df.loc[df["geometry"].apply(lambda x: x.geometryType()) == "MultiLineString","geometry"] = \
        df.loc[df["geometry"].apply(lambda x: x.geometryType()) == "MultiLineString","geometry"].apply(lambda x: x[0])

    # Get rid of erroneous 1MV and low power line voltages
    df = df[(df["VOLTAGE"] < 1000) & (df["VOLTAGE"] >= 200)]

    df["obj"] = df.apply(bezpy.tl.TransmissionLine, axis=1)
    df["length"] = df.obj.apply(lambda x: x.length)

    print("Starting delaunay calculations which can take ~20 minutes")
    t1 = time.time()
    df.obj.apply(lambda x: x.set_delaunay_weights(site_xys, use_gnomic=False))
    print(f"Done filling interpolation weights: {time.time() - t1} s".format())
    # Create the pickle object so we can load faster next time
    df.to_pickle("transmission_line_objects.pkl")
    return df


def calc_voltages(df_lines, ds_E):
    """Calculate the voltages along the lines for the given electric field time-series"""
    ntimes = len(ds_E["time"])
    n_trans_lines = len(df)
    arr_delaunay = np.zeros(shape=(ntimes, n_trans_lines))
    e_fields = ds_E["E"].to_numpy()
    # Iterate over all transmission lines to do the integration
    for i, t_line in enumerate(df_lines.obj):
        arr_delaunay[:, i] = t_line.calc_voltages(e_fields, how='delaunay')
    
    # We need to add a new coordinate, the "line" which corresponds to the index
    # in the line dataframe
    ds_E = ds_E.assign_coords({"line": np.arange(n_trans_lines)})
    # gic_proxy = V / R = V / (resistivity * line_length)
    rho_line = 0.1  # Ohms/km (Grigsby 2007)
    ds_E = ds_E.assign({"V": (("time", "line"), arr_delaunay),
                        "I": (("time", "line"), arr_delaunay / ((rho_line * df_lines["length"].to_numpy())[np.newaxis, :]))})
    return ds_E

In [9]:
df = get_transmission_lines()

try:
    mage_ds = xr.load_dataset("mage_data.nc")
except FileNotFoundError:
    # Create the data and save it
    mage_ds = read_mage("finBoo.deltab.h5")
    mage_mapping = create_mapping(model_sites_sorted, mage_ds)
    mage_ds = calc_e(mage_mapping, mage_ds)
    mage_ds = calc_voltages(df, mage_ds)
    mage_ds.to_netcdf("mage_data.nc")

## Plotting

In [13]:
# Generate a regular grid of (x, y) pairs
gridx, gridy = np.meshgrid(sorted(np.unique(site_xys[:, 0])),  # lons, lats
                           sorted(np.unique(site_xys[:, 1])))
gridxflat = gridx.ravel()
gridyflat = gridy.ravel()

# Generate a list of index transfer functions to save and reuse later
idxs = np.zeros(nsites, dtype=int)
for i in range(nsites):
    idxs[i] = np.argwhere((site_xys[i, 0] == gridxflat)
                          & (site_xys[i, 1] == gridyflat))[0][0]

def fill_mesh(data):
    """Fills the mesh with the data in the proper locations"""
    output = np.full(gridx.ravel().size, np.nan, dtype=data.dtype)
    output[idxs] = data
    return output.reshape(gridx.shape)

In [14]:
from matplotlib import animation
import matplotlib as mpl
from IPython.display import HTML
mpl.rc('animation', html='html5')

import cartopy.crs as ccrs
import cartopy.feature as cfeature

scale = '10m'
land = cfeature.NaturalEarthFeature('physical', 'land', scale,
                                    edgecolor='face',
                                    facecolor=cfeature.COLORS['land'])
coast = cfeature.NaturalEarthFeature(category='physical', scale=scale,
                                     edgecolor='k',
                                     facecolor='none', name='coastline')
ocean = cfeature.NaturalEarthFeature(
        category='physical',
        name='ocean',
        scale=scale,
        facecolor=cfeature.COLORS['water'])
rivers = cfeature.NaturalEarthFeature(
        category='physical',
        name='rivers_lake_centerlines',
        scale=scale,
        facecolor=cfeature.COLORS['water'],
        edgecolor='face')
lakes = cfeature.NaturalEarthFeature(
        category='physical',
        name='lakes',
        scale=scale,
        facecolor=cfeature.COLORS['water'],
        edgecolor='face')

states = cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_1_states_provinces_lines',
        scale=scale,
        facecolor='none',
        edgecolor='k')
countries = cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_0_countries',
        scale=scale,
        facecolor='none',
        edgecolor='k')

def get_intersections(df, bbox):
    spatial_index = df.sindex

    x0, x1, y0, y1 = bbox
    polygon = shapely.geometry.Polygon([[x0, y0], [x1, y0], [x1, y1], [x0, y1]])

    possible_matches_index = list(spatial_index.intersection(polygon.bounds))
    possible_matches = df.iloc[possible_matches_index]
    precise_matches = possible_matches[possible_matches.intersects(polygon)]
    return precise_matches

def symlog(x):
    """ Returns the symmetric log10 value"""
    # Add one to avoid low values from being shifted lower, we don't
    # need exact values here
    return np.sign(x) * np.log10(1 + np.abs(x))

# line widths go from 0.5 -> 2, equating to 1 -> 1000 km on log scale?
line_length_min = 10
line_length_max = 1000
line_width_min = 0.25
line_width_max = 2

# Set up the equations
def calc_line_width(x, log_scale=False):
    if log_scale:
        line_width = (np.log10(x) - np.log10(line_length_min))/ \
        (np.log10(line_length_max) - np.log10(line_length_min)) * \
        (line_width_max - line_width_min) + line_width_min
    else:
        line_width = (x - line_length_min)/ \
        (line_length_max - line_length_min) * \
        (line_width_max - line_width_min) + line_width_min
    return np.clip(line_width, a_min=line_width_min, a_max=line_width_max)


bbox = (np.min(site_xys[:, 0]), np.max(site_xys[:, 0]),
        np.min(site_xys[:, 1]), np.max(site_xys[:, 1]))

def setup_axes(ax):
    """Setup an axes with the proper background, features, and extent."""
    ax.set_extent(bbox, ccrs.PlateCarree())
#     ax.add_feature(coast)
    ax.set_facecolor(cfeature.COLORS["water"])
    ax.add_feature(cfeature.LAND, color=(0.8, 0.8, 0.8, 1))
    ax.add_feature(states, alpha=0.8)
    ax.add_feature(countries)

In [15]:
proj_data = ccrs.PlateCarree()

B_norm = mpl.colors.Normalize(0, 1000)
B_cmap = mpl.cm.get_cmap("viridis")

E_norm = mpl.colors.Normalize(0, 1000)
E_cmap = mpl.cm.get_cmap("viridis")

I_norm = mpl.colors.Normalize(0, 10)
I_cmap = plt.get_cmap("plasma").copy()
I_cmap.set_bad(alpha=0.)

line_coordinates = [np.array(linestring)[:, :2] for linestring in df['geometry']]
line_widths = calc_line_width(df["length"], log_scale=True)


def plot_quantity(ax, ds, t=0, quantity="B"):
    """Plots the requested quantity from the dataset on the axes"""
    if quantity == "I":
        # We need to add a LineCollection
        coll = mpl.collections.LineCollection(line_coordinates)
        coll.set_array(np.abs(ds["I"].to_numpy()[t, :]))
        coll.set_cmap(I_cmap)
        coll.set_norm(I_norm)
        coll.set_transform(proj_data)
        coll.set_linewidths(line_widths)
        ax.add_collection(coll)
        return coll

    elif quantity == "B":
        norm = B_norm
        cmap = B_cmap
    elif quantity == "E":
        norm = E_norm
        cmap = E_cmap

    arr = ds[quantity].to_numpy()
    arr = np.sqrt(arr[t, :, 0]**2 + arr[t, :, 1]**2)
    mesh = ax.pcolormesh(gridx, gridy, fill_mesh(arr),
                         transform=proj_data, cmap=cmap, norm=norm, alpha=0.5)
    return mesh


def update_quantity(coll, ds, t=0, quantity="B"):
    if quantity == "I":
        # Line collection, so just set_array with the new data
        coll.set_array(np.abs(ds[quantity].to_numpy()[t, :]))
        return
    arr = ds[quantity].to_numpy()
    arr = np.sqrt(arr[t, :, 0]**2 + arr[t, :, 1]**2)
    coll.set_array(fill_mesh(arr).ravel())

## B + E + TransmissionLine GIC

In [16]:
projection = ccrs.Mercator()
fig, (ax_Bfield, ax_Efield, ax_gic) = plt.subplots(figsize=(8, 12),
                                                   nrows=3,
                                                   constrained_layout=True,
                                                   subplot_kw=dict(projection=projection),
                                                   dpi=200)

for ax in [ax_Bfield, ax_Efield, ax_gic]:
    setup_axes(ax)

t = 0
ds = mage_ds
model_name = "MAGE"
times = ds["time"].dt.strftime('%Y/%m/%d %H:%M').to_numpy()
title = ax_Bfield.set_title(f"{model_name}\n{times[t]}")

B_mesh = plot_quantity(ax_Bfield, ds, t=t, quantity="B")
E_mesh = plot_quantity(ax_Efield, ds, t=t, quantity="E")
I_coll = plot_quantity(ax_gic, ds, t=t, quantity="I")

cbar = fig.colorbar(ax=ax_Bfield, mappable=B_mesh, orientation='vertical')
cbar.set_label('Magnetic Field ($nT$)')

cbar = fig.colorbar(ax=ax_Efield, mappable=E_mesh, orientation='vertical')
cbar.set_label('Electric Field ($mV/km$)')

cbar = fig.colorbar(ax=ax, mappable=I_coll, orientation='vertical')
cbar.set_label('GIC Proxy ($A$)')

fig.canvas.draw()
fig.canvas.draw()
fig.set_constrained_layout(False)

def animate(t):
    title.set_text(f"{model_name}\n{times[t]}")
    update_quantity(B_mesh, ds, t=t, quantity="B")
    update_quantity(E_mesh, ds, t=t, quantity="E")
    update_quantity(I_coll, ds, t=t, quantity="I")
    
anim = animation.FuncAnimation(fig, animate, frames=range(len(times)))
anim.save(f"{model_name}-Halloween.mp4", dpi=200)
plt.close()
# HTML(anim.to_html5_video())

In [ ]:
projection = ccrs.Mercator()
fig, (ax_Bfield, ax_Efield, ax_gic) = plt.subplots(figsize=(8, 12),
                                                   nrows=3,
                                                   constrained_layout=True,
                                                   subplot_kw=dict(projection=projection),
                                                   dpi=200)

for ax in [ax_Bfield, ax_Efield, ax_gic]:
    setup_axes(ax)

t = 0
ds = swpc_ds
model_name = "SWPC"
times = ds["time"].dt.strftime('%Y/%m/%d %H:%M').to_numpy()
title = ax_Bfield.set_title(f"{model_name}\n{times[t]}")

B_mesh = plot_quantity(ax_Bfield, ds, t=t, quantity="B")
E_mesh = plot_quantity(ax_Efield, ds, t=t, quantity="E")
I_coll = plot_quantity(ax_gic, ds, t=t, quantity="I")

cbar = fig.colorbar(ax=ax_Bfield, mappable=B_mesh, orientation='vertical')
cbar.set_label('Magnetic Field ($nT$)')

cbar = fig.colorbar(ax=ax_Efield, mappable=E_mesh, orientation='vertical')
cbar.set_label('Electric Field ($mV/km$)')

cbar = fig.colorbar(ax=ax, mappable=I_coll, orientation='vertical')
cbar.set_label('GIC Proxy ($A$)')

fig.canvas.draw()
fig.canvas.draw()
fig.set_constrained_layout(False)

def animate(t):
    title.set_text(f"{model_name}\n{times[t]}")
    update_quantity(B_mesh, ds, t=t, quantity="B")
    update_quantity(E_mesh, ds, t=t, quantity="E")
    update_quantity(I_coll, ds, t=t, quantity="I")
    
anim = animation.FuncAnimation(fig, animate, frames=range(len(times)))
anim.save(f"{model_name}-Halloween.mp4", dpi=200)
plt.close()
# HTML(anim.to_html5_video())

In [ ]:
projection = ccrs.Mercator()
fig, (ax_Bfield, ax_Efield, ax_gic) = plt.subplots(figsize=(8, 12),
                                                   nrows=3,
                                                   constrained_layout=True,
                                                   subplot_kw=dict(projection=projection),
                                                   dpi=200)

for ax in [ax_Bfield, ax_Efield, ax_gic]:
    setup_axes(ax)

t = 0
ds = umich_ds
model_name = "UMICH"
times = ds["time"].dt.strftime('%Y/%m/%d %H:%M').to_numpy()
title = ax_Bfield.set_title(f"{model_name}\n{times[t]}")

B_mesh = plot_quantity(ax_Bfield, ds, t=t, quantity="B")
E_mesh = plot_quantity(ax_Efield, ds, t=t, quantity="E")
I_coll = plot_quantity(ax_gic, ds, t=t, quantity="I")

cbar = fig.colorbar(ax=ax_Bfield, mappable=B_mesh, orientation='vertical')
cbar.set_label('Magnetic Field ($nT$)')

cbar = fig.colorbar(ax=ax_Efield, mappable=E_mesh, orientation='vertical')
cbar.set_label('Electric Field ($mV/km$)')

cbar = fig.colorbar(ax=ax, mappable=I_coll, orientation='vertical')
cbar.set_label('GIC Proxy ($A$)')

fig.canvas.draw()
fig.canvas.draw()
fig.set_constrained_layout(False)

def animate(t):
    title.set_text(f"{model_name}\n{times[t]}")
    update_quantity(B_mesh, ds, t=t, quantity="B")
    update_quantity(E_mesh, ds, t=t, quantity="E")
    update_quantity(I_coll, ds, t=t, quantity="I")
    
anim = animation.FuncAnimation(fig, animate, frames=range(len(times)))
anim.save(f"{model_name}-Halloween.mp4", dpi=200)
plt.close()
# HTML(anim.to_html5_video())

In [ ]:
projection = ccrs.Mercator()
fig, (ax_Bfield, ax_Efield, ax_gic) = plt.subplots(figsize=(8, 12),
                                                   nrows=3,
                                                   constrained_layout=True,
                                                   subplot_kw=dict(projection=projection),
                                                   dpi=200)

for ax in [ax_Bfield, ax_Efield, ax_gic]:
    setup_axes(ax)

t = 0
ds = mage_ds
model_name = "MAGE"
times = ds["time"].dt.strftime('%Y/%m/%d %H:%M').to_numpy()
title = ax_Bfield.set_title(f"{model_name}\n{times[t]}")

B_mesh = plot_quantity(ax_Bfield, ds, t=t, quantity="B")
E_mesh = plot_quantity(ax_Efield, ds, t=t, quantity="E")
I_coll = plot_quantity(ax_gic, ds, t=t, quantity="I")

cbar = fig.colorbar(ax=ax_Bfield, mappable=B_mesh, orientation='vertical')
cbar.set_label('Magnetic Field ($nT$)')

cbar = fig.colorbar(ax=ax_Efield, mappable=E_mesh, orientation='vertical')
cbar.set_label('Electric Field ($mV/km$)')

cbar = fig.colorbar(ax=ax, mappable=I_coll, orientation='vertical')
cbar.set_label('GIC Proxy ($A$)')

fig.canvas.draw()
fig.canvas.draw()
fig.set_constrained_layout(False)

def animate(t):
    title.set_text(f"{model_name}\n{times[t]}")
    update_quantity(B_mesh, ds, t=t, quantity="B")
    update_quantity(E_mesh, ds, t=t, quantity="E")
    update_quantity(I_coll, ds, t=t, quantity="I")
    
anim = animation.FuncAnimation(fig, animate, frames=range(len(times)))
anim.save(f"{model_name}-Halloween.mp4", dpi=200)
plt.close()
# HTML(anim.to_html5_video())